In [1]:
import torch
import torch.nn as nn
from torch._C import device
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import torch.optim as optim
import time
from torch import optim
from torch.optim import lr_scheduler
from tqdm import tqdm
from scipy import interpolate
plt.style.use(["classic"])
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch.nn.functional as F

In [2]:
def vis_L(epoch, x, y):
    if epoch % 50 != 0: return
    plt.plot(x.cpu().detach().numpy(), y.cpu().detach().numpy(), "k--",label="MLP")
    plt.xlabel(r"$r$")
    plt.ylabel(r"$L$")
    plt.legend(loc="best")
    plt.savefig("pic/r_vs_L_epoch%s.jpg"%epoch,bbox_inches = "tight")
    plt.close()
def vis_W(epoch, x, y):
    if epoch % 50 != 0: return
    plt.plot(x.cpu().detach().numpy(), y.cpu().detach().numpy(), "k--",label="MLP")
    plt.xlabel(r"$r$")
    plt.ylabel(r"$W$")
    plt.legend(loc="best")
    plt.savefig("pic/r_vs_W_epoch%s.jpg"%epoch,bbox_inches = "tight")
    plt.close()
def vis_LE(epoch, x, y1, y2):
    if epoch % 50 != 0: return
    plt.plot(x.cpu().detach().numpy(),y1.cpu().detach().numpy(),"k--",label="MLP")
    plt.plot(x.cpu().detach().numpy(),y2.cpu().detach().numpy(),"r--",label="lattice")
    plt.xlabel(r"$L$")
    plt.ylabel(r"$E$")
    plt.legend(loc="best")
    plt.savefig("pic/L_vs_E_epoch%s.jpg"%epoch,bbox_inches = "tight")
    plt.close()  
def vis_x(epoch, x, y1, y2):
    if epoch % 50 != 0: return
    plt.plot(x.cpu().detach().numpy(),y1.cpu().detach().numpy(),"k--",label="MLP")
#     plt.plot(x.cpu().detach().numpy(),y2.cpu().detach().numpy(),"r--",label="exp(0.4r)")
    plt.xlabel(r"$r$")
    plt.ylabel(r"$w$")
    plt.legend(loc="best")
    plt.savefig("pic/x_vs_r_epoch%s.jpg"%epoch,bbox_inches = "tight")
    plt.close()     

In [3]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3 ,output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)
        self.fc3 = nn.Linear(hidden_size2, hidden_size3)
        self.fc4 = nn.Linear(hidden_size3, output_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x)) 
        x = torch.relu(self.fc2(x)) 
        x = torch.relu(self.fc3(x)) 
#         x = torch.exp(self.fc4(x))  # 使用Softplus确保输出非负 
        x = F.softplus(self.fc4(x))
        return x

# 公式的

In [11]:
def DNN_train(rmin, rmax, num_sample, learning_rate, epochs, path, device, func_LE):
    """
    rmin, rmax: the min and max of r
    num_sample: the num of sample
    func_LE: L as a function of HQ energy
    note: using the DNN to represent the x, w = x/r^2
    """
    x_func = Net(1,64,128,64,1).to(device) 
#     x_func = Net(1,8,128,1).to(device)
#     opt = optim.AdamW([{"params":x_func.parameters()}], lr = learning_rate)
    opt = optim.AdamW([{"params":x_func.parameters()}], lr = learning_rate,betas=(0.9,0.999))
#     opt = optim.SGD([{"params": x_func.parameters()}], lr=learning_rate)

    lr_step_size =1000
    schedular = lr_scheduler.StepLR(opt, step_size=lr_step_size, gamma=1) 
    
    x_func.train()
    
    ########## loss
    def loss_a(r):
        
        g = 0.176
        r.requires_grad = True
        r_min = 0.01
        deg = 100
        xk, wk = np.polynomial.legendre.leggauss(deg)

        xk = torch.from_numpy(xk).float().to(device)
        wk = torch.from_numpy(wk).float().to(device)
        
        x_f = x_func(r) 
        wr_func = x_f / r**2

        L_dis  = torch.tensor([]).to(device)  
        m1  = torch.tensor([]).to(device)  

        for idx in range(len(r)):
            """
            gauss legendre, deg = 50, xn is the node of r, and wn is the node of weight
            calculating the integration part of distance L and HQ energy
            """
            xn = (r[idx] - r_min) * 0.5 * xk +(r[idx] + r_min) * 0.5
            wn = wk * (r[idx] - r_min) * 0.5

            xn = xn.reshape(-1,1)
            wn = wn.reshape(-1,1)  

            xn = xn.to(torch.float32)
            wn = wn.to(torch.float32)
            

            x_fn = x_func(xn) 
            w_rn = x_fn / xn**2
            hn = torch.tensor([0.0], requires_grad=True, device=device, dtype=torch.float).reshape(-1, 1).to(device)
            x_fn0 = x_func(hn) 
            x_fn0d = torch.autograd.grad(outputs=x_fn0, inputs=hn, create_graph=True)[0]
            w_r0 = x_func(r[idx])/r[idx] ** 2
            
            Inter1 = 2  * 0.1973 * wn * (w_r0 ** 2 / (w_rn ** 2 - w_r0 ** 2)) ** 0.5
            Inter2 = wn * w_rn * (1 + w_r0 ** 2 / (w_rn ** 2 - w_r0 ** 2)) ** 0.5  - wn * x_fn0 / xn ** 2 - wn * x_fn0d / xn 
        
            p1 = torch.sum(Inter1)
            p2 = torch.sum(Inter2)
            
            p1 = p1.reshape(-1,1)
            p2 = p2.reshape(-1,1)
            
            L_dis = torch.cat((L_dis,p1),0)
            m1 = torch.cat((m1,p2),0) 
        HQ_pred = 2 * g * m1 - 2 * g * x_fn0 / r + 2*g* x_fn0d*torch.log(r)
#         HQ_true = E_t.detach().cpu().numpy()
#         L_true = L_t.detach().cpu().numpy()
        HQ_true = func_LE(L_dis.detach().cpu().numpy())
        HQ_true = torch.FloatTensor(HQ_true).reshape(-1,1).to(device)
#         print("L",L_dis)
#         L_true = torch.FloatTensor(L_true).reshape(-1,1).to(device)
#         loss_mae = nn.MSELoss()
#         loss_mae = nn.KLDivLoss()
        constraint_loss = (x_fn0 - 1.0) ** 2
        loss_mae = nn.L1Loss()
        loss_1 = loss_mae(HQ_pred, HQ_true).to(device)

        input_tensor = torch.tensor([0.], device=device)
        target_tensor = torch.tensor([1], device=device)
        loss_2 = loss_mae(x_func(input_tensor), target_tensor)

#         loss_2 = loss_mae(x_func(torch.tensor([0.])), torch.tensor([1])).to(device)
        loss_tot = loss_1 + loss_2+constraint_loss
        
        return loss_tot, r, wr_func, L_dis, HQ_pred, HQ_true, x_f

    tic = time.time()
    Loss_list = []
    Loss_mean = 1e5
    
    r = np.linspace(rmin, rmax, num_sample)
    r = torch.FloatTensor(r).reshape(-1,1).to(device) 
    
    # Each step of the training process:
    for epoch in range(epochs):
        Loss = []
        x_func.zero_grad()        
        loss, r, wr_func, L_dis, HQ_pred, HQ_true, x_f = loss_a(r)
        
        # plot same fig during training
#         vis_L(epoch, r, L_dis)
        vis_x(epoch, r, x_f, torch.exp(0.450*r**2))
#         vis_LE(epoch, r, HQ_pred, HQ_true)
        vis_LE(epoch,L_dis,HQ_pred,HQ_true)
#         vis_W(epoch, r, wr_func)
 
        # Back propagation and optim
        loss.backward()
        opt.step()
        schedular.step()
                
        
        lr = schedular.get_last_lr()[0]

        # print the loss value
        Loss.append(loss.item())
        print('Train Epoch:{} learning rate:{:.4e}, Loss_tot:{:.4f},'.format(epoch+1, lr, loss.item()))
              
        Loss = np.array(Loss)
        
        # 检测Loss是否为NaN
        if torch.isnan(loss):
            print("Loss is NaN. Saving the previous model...")
#             torch.save(checkpoint1, path+"x_func_model1.pt")
            break
            
        # check the loss and save the model        
        if np.mean(Loss) < Loss_mean:
        
            checkpoint1 = {
                    'epoch': epoch + 1,
                    'state_dict': x_func.state_dict(),
                    'optimizer': opt.state_dict(),
                    'loss': loss.item()
                    }
            
            torch.save(checkpoint1,path+"x_func_model.pt")
            print("save model")
            Loss_mean = np.mean(Loss)
            
            torch.save(L_dis, path + 'L_dist.pt')
            torch.save(HQ_pred, path + 'HQ_pred.pt')
            torch.save(x_f, path + 'x_f.pt')
            torch.save(r,path + 'r.pt')
        
        # 大于1000的时候每次都保存模型
        if epoch > 1000:
            model_state = {
                'epoch': epoch + 1,
                'state_dict': x_func.state_dict(),
                'optimizer': opt.state_dict(),
                'loss': loss.item()
            }
            torch.save(model_state, "./model/log/" + f"x_func_model_{epoch + 1}.pt")
            print(f"Save model {epoch + 1}")
            
    toc = time.time()
    trainTime = toc - tic
    print("Training time = ", trainTime)
    np.save(path + 'Loss_all',np.array(Loss_list))

In [12]:
# builf the folder
def mkdir(path):

    folder = os.path.exists(path)

    if not folder:                   
        os.makedirs(path)            
        print ("---  new folder...  ---")
        print ("---  OK  ---")

    else:
        print ("---  There is this folder!  ---")
mkdir('data')
mkdir('model')
mkdir('pic')

---  There is this folder!  ---
---  There is this folder!  ---
---  There is this folder!  ---


In [13]:
def func_LE_wrapper(L):
    E =0.8404*L - 0.0866/L +0.1033
    return E

In [14]:
# some parameters
epochs = 10001
path_data = "./data/"
path_model = "./model/"
path_pic = "./pic/" 
learning_rate = 1e-5
rmin = 0.05
rmax = 3
n = 150
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DNN_train(rmin, rmax, n,learning_rate, epochs, path_model, device,func_LE_wrapper)

Train Epoch:1 learning rate:1.0000e-05, Loss_tot:0.7646,
save model
Train Epoch:2 learning rate:1.0000e-05, Loss_tot:0.7641,
save model
Train Epoch:3 learning rate:1.0000e-05, Loss_tot:0.7636,
save model
Train Epoch:4 learning rate:1.0000e-05, Loss_tot:0.7632,
save model
Train Epoch:5 learning rate:1.0000e-05, Loss_tot:0.7628,
save model
Train Epoch:6 learning rate:1.0000e-05, Loss_tot:0.7624,
save model
Train Epoch:7 learning rate:1.0000e-05, Loss_tot:0.7620,
save model
Train Epoch:8 learning rate:1.0000e-05, Loss_tot:0.7616,
save model
Train Epoch:9 learning rate:1.0000e-05, Loss_tot:0.7612,
save model
Train Epoch:10 learning rate:1.0000e-05, Loss_tot:0.7608,
save model
Train Epoch:11 learning rate:1.0000e-05, Loss_tot:0.7604,
save model
Train Epoch:12 learning rate:1.0000e-05, Loss_tot:0.7600,
save model
Train Epoch:13 learning rate:1.0000e-05, Loss_tot:0.7596,
save model
Train Epoch:14 learning rate:1.0000e-05, Loss_tot:0.7591,
save model
Train Epoch:15 learning rate:1.0000e-05, Lo

Train Epoch:123 learning rate:1.0000e-05, Loss_tot:0.7282,
save model
Train Epoch:124 learning rate:1.0000e-05, Loss_tot:0.7279,
save model
Train Epoch:125 learning rate:1.0000e-05, Loss_tot:0.7276,
save model
Train Epoch:126 learning rate:1.0000e-05, Loss_tot:0.7273,
save model
Train Epoch:127 learning rate:1.0000e-05, Loss_tot:0.7269,
save model
Train Epoch:128 learning rate:1.0000e-05, Loss_tot:0.7266,
save model
Train Epoch:129 learning rate:1.0000e-05, Loss_tot:0.7263,
save model
Train Epoch:130 learning rate:1.0000e-05, Loss_tot:0.7259,
save model
Train Epoch:131 learning rate:1.0000e-05, Loss_tot:0.7256,
save model
Train Epoch:132 learning rate:1.0000e-05, Loss_tot:0.7253,
save model
Train Epoch:133 learning rate:1.0000e-05, Loss_tot:0.7249,
save model
Train Epoch:134 learning rate:1.0000e-05, Loss_tot:0.7246,
save model
Train Epoch:135 learning rate:1.0000e-05, Loss_tot:0.7242,
save model
Train Epoch:136 learning rate:1.0000e-05, Loss_tot:0.7239,
save model
Train Epoch:137 lear

Train Epoch:245 learning rate:1.0000e-05, Loss_tot:0.6905,
Train Epoch:246 learning rate:1.0000e-05, Loss_tot:0.6907,
Train Epoch:247 learning rate:1.0000e-05, Loss_tot:0.6904,
Train Epoch:248 learning rate:1.0000e-05, Loss_tot:0.6896,
Train Epoch:249 learning rate:1.0000e-05, Loss_tot:0.6892,
Train Epoch:250 learning rate:1.0000e-05, Loss_tot:0.6894,
Train Epoch:251 learning rate:1.0000e-05, Loss_tot:0.6890,
Train Epoch:252 learning rate:1.0000e-05, Loss_tot:0.6886,
Train Epoch:253 learning rate:1.0000e-05, Loss_tot:0.6878,
save model
Train Epoch:254 learning rate:1.0000e-05, Loss_tot:0.6874,
save model
Train Epoch:255 learning rate:1.0000e-05, Loss_tot:0.6884,
Train Epoch:256 learning rate:1.0000e-05, Loss_tot:0.6859,
save model
Train Epoch:257 learning rate:1.0000e-05, Loss_tot:0.6877,
Train Epoch:258 learning rate:1.0000e-05, Loss_tot:0.6875,
Train Epoch:259 learning rate:1.0000e-05, Loss_tot:0.6872,
Train Epoch:260 learning rate:1.0000e-05, Loss_tot:0.6869,
Train Epoch:261 learnin

Train Epoch:376 learning rate:1.0000e-05, Loss_tot:0.6547,
save model
Train Epoch:377 learning rate:1.0000e-05, Loss_tot:0.6543,
save model
Train Epoch:378 learning rate:1.0000e-05, Loss_tot:0.6524,
save model
Train Epoch:379 learning rate:1.0000e-05, Loss_tot:0.6536,
Train Epoch:380 learning rate:1.0000e-05, Loss_tot:0.6518,
save model
Train Epoch:381 learning rate:1.0000e-05, Loss_tot:0.6530,
Train Epoch:382 learning rate:1.0000e-05, Loss_tot:0.6527,
Train Epoch:383 learning rate:1.0000e-05, Loss_tot:0.6524,
Train Epoch:384 learning rate:1.0000e-05, Loss_tot:0.6521,
Train Epoch:385 learning rate:1.0000e-05, Loss_tot:0.6517,
save model
Train Epoch:386 learning rate:1.0000e-05, Loss_tot:0.6514,
save model
Train Epoch:387 learning rate:1.0000e-05, Loss_tot:0.6510,
save model
Train Epoch:388 learning rate:1.0000e-05, Loss_tot:0.6508,
save model
Train Epoch:389 learning rate:1.0000e-05, Loss_tot:0.6491,
save model
Train Epoch:390 learning rate:1.0000e-05, Loss_tot:0.6488,
save model
Train

Train Epoch:504 learning rate:1.0000e-05, Loss_tot:0.6163,
save model
Train Epoch:505 learning rate:1.0000e-05, Loss_tot:0.6169,
Train Epoch:506 learning rate:1.0000e-05, Loss_tot:0.6178,
Train Epoch:507 learning rate:1.0000e-05, Loss_tot:0.6176,
Train Epoch:508 learning rate:1.0000e-05, Loss_tot:0.6173,
Train Epoch:509 learning rate:1.0000e-05, Loss_tot:0.6170,
Train Epoch:510 learning rate:1.0000e-05, Loss_tot:0.6166,
Train Epoch:511 learning rate:1.0000e-05, Loss_tot:0.6152,
save model
Train Epoch:512 learning rate:1.0000e-05, Loss_tot:0.6149,
save model
Train Epoch:513 learning rate:1.0000e-05, Loss_tot:0.6158,
Train Epoch:514 learning rate:1.0000e-05, Loss_tot:0.6155,
Train Epoch:515 learning rate:1.0000e-05, Loss_tot:0.6152,
Train Epoch:516 learning rate:1.0000e-05, Loss_tot:0.6149,
save model
Train Epoch:517 learning rate:1.0000e-05, Loss_tot:0.6136,
save model
Train Epoch:518 learning rate:1.0000e-05, Loss_tot:0.6124,
save model
Train Epoch:519 learning rate:1.0000e-05, Loss_to

...

Train Epoch:9509 learning rate:1.0000e-05, Loss_tot:0.0853,
save model
Save model 9509
Train Epoch:9510 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9510
Train Epoch:9511 learning rate:1.0000e-05, Loss_tot:0.0855,
Save model 9511
Train Epoch:9512 learning rate:1.0000e-05, Loss_tot:0.0856,
Save model 9512
Train Epoch:9513 learning rate:1.0000e-05, Loss_tot:0.0855,
Save model 9513
Train Epoch:9514 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9514
Train Epoch:9515 learning rate:1.0000e-05, Loss_tot:0.0853,
Save model 9515
Train Epoch:9516 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9516
Train Epoch:9517 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9517
Train Epoch:9518 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9518
Train Epoch:9519 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9519
Train Epoch:9520 learning rate:1.0000e-05, Loss_tot:0.0853,
save model
Save model 9520
Train Epoch:9521 learning rate:1.0000e-05, Loss_tot:0.0853,
Save m

Train Epoch:9615 learning rate:1.0000e-05, Loss_tot:0.0850,
save model
Save model 9615
Train Epoch:9616 learning rate:1.0000e-05, Loss_tot:0.0850,
save model
Save model 9616
Train Epoch:9617 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9617
Train Epoch:9618 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9618
Train Epoch:9619 learning rate:1.0000e-05, Loss_tot:0.1609,
Save model 9619
Train Epoch:9620 learning rate:1.0000e-05, Loss_tot:0.1608,
Save model 9620
Train Epoch:9621 learning rate:1.0000e-05, Loss_tot:0.1604,
Save model 9621
Train Epoch:9622 learning rate:1.0000e-05, Loss_tot:0.1600,
Save model 9622
Train Epoch:9623 learning rate:1.0000e-05, Loss_tot:0.1595,
Save model 9623
Train Epoch:9624 learning rate:1.0000e-05, Loss_tot:0.1588,
Save model 9624
Train Epoch:9625 learning rate:1.0000e-05, Loss_tot:0.1580,
Save model 9625
Train Epoch:9626 learning rate:1.0000e-05, Loss_tot:0.1571,
Save model 9626
Train Epoch:9627 learning rate:1.0000e-05, Loss_tot:0.1563,
Save m

Train Epoch:9723 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9723
Train Epoch:9724 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9724
Train Epoch:9725 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9725
Train Epoch:9726 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9726
Train Epoch:9727 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9727
Train Epoch:9728 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9728
Train Epoch:9729 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9729
Train Epoch:9730 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9730
Train Epoch:9731 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9731
Train Epoch:9732 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9732
Train Epoch:9733 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9733
Train Epoch:9734 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9734
Train Epoch:9735 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9735
Train Epoch:

Train Epoch:9830 learning rate:1.0000e-05, Loss_tot:0.0849,
Save model 9830
Train Epoch:9831 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9831
Train Epoch:9832 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9832
Train Epoch:9833 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9833
Train Epoch:9834 learning rate:1.0000e-05, Loss_tot:0.0849,
Save model 9834
Train Epoch:9835 learning rate:1.0000e-05, Loss_tot:0.0849,
Save model 9835
Train Epoch:9836 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9836
Train Epoch:9837 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9837
Train Epoch:9838 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9838
Train Epoch:9839 learning rate:1.0000e-05, Loss_tot:0.0848,
save model
Save model 9839
Train Epoch:9840 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9840
Train Epoch:9841 learning rate:1.0000e-05, Loss_tot:0.0848,
Save model 9841
Train Epoch:9842 learning rate:1.0000e-05, Loss_tot:0.0848,
save model
Save m

Train Epoch:9936 learning rate:1.0000e-05, Loss_tot:0.0869,
Save model 9936
Train Epoch:9937 learning rate:1.0000e-05, Loss_tot:0.0866,
Save model 9937
Train Epoch:9938 learning rate:1.0000e-05, Loss_tot:0.0863,
Save model 9938
Train Epoch:9939 learning rate:1.0000e-05, Loss_tot:0.0861,
Save model 9939
Train Epoch:9940 learning rate:1.0000e-05, Loss_tot:0.0858,
Save model 9940
Train Epoch:9941 learning rate:1.0000e-05, Loss_tot:0.0857,
Save model 9941
Train Epoch:9942 learning rate:1.0000e-05, Loss_tot:0.0855,
Save model 9942
Train Epoch:9943 learning rate:1.0000e-05, Loss_tot:0.0854,
Save model 9943
Train Epoch:9944 learning rate:1.0000e-05, Loss_tot:0.0853,
Save model 9944
Train Epoch:9945 learning rate:1.0000e-05, Loss_tot:0.0852,
Save model 9945
Train Epoch:9946 learning rate:1.0000e-05, Loss_tot:0.0851,
Save model 9946
Train Epoch:9947 learning rate:1.0000e-05, Loss_tot:0.0850,
Save model 9947
Train Epoch:9948 learning rate:1.0000e-05, Loss_tot:0.0850,
Save model 9948
Train Epoch: